In [5]:
# Isaac G Callison
# 6350-001
# Project 3
# Due 2/11/2020

In [6]:
# imports
import random
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk import classify
from nltk.metrics import *
import collections

#set(stopwords.words('and'))

In [7]:
# pull in Sotu from file 
sotu_file = 'sotu.txt'
pres_by_party = 'presidentsbyparty.txt'

repub_list = [] ; dem_list = []

with open(pres_by_party) as prez:
    cl = prez.readline()
    cl = cl.strip()
    if (cl == 'Republican'):
        while cl != 'Democrat':
            cl = prez.readline()
            cl = cl.strip()
            if len(cl) > 1:
                repub_list.append(cl)
    repub_list = repub_list[:-1]
    for line in prez:
        if len(line) > 1:
            line = line.strip()
            dem_list.append(line)
     
#sotu = open(sotu_file,'r')
# shift + tab shows documenation. 


In [ ]:
testFile = open("test.txt","w+")

with open(sotu_file) as sf:
    line = sf.readline()
    while line:
        line = line.strip()
        if '@' in line:
            line = line.split('@',4)
            line = line[1]
            if line in repub_list:
                text = sf.readline()
                text = text.strip()
                while '@' not in text:
                    #print(text)
                    testFile.write(text)
                    text = sf.readline()
            
        line = sf.readline()
        
testFile.close()


In [ ]:
# Clean Sotu, truncate to republican and dem and normalize data 

In [66]:
# write out to file, should have just repubs and demrats


In [23]:
# Implement features - how to differentiate from each other? 

In [24]:
# USE stopwords??
#stop_words = set(stopwords.words('and'))

In [25]:
# Run classifier